# Configure the SFT Model with LoRA

针对 “你是谁” 这一问题对模型进行微调，同时需采取措施避免模型发生灾难性遗忘，确保 “xxx 是谁” 这类相似问题的原有答案不被覆盖。

参考： https://docs.unsloth.ai/

In [27]:
from unsloth import FastModel
import torch
torch.cuda.empty_cache() 

In [28]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_data_formats
from transformers import TextStreamer


In [29]:
IS_TRAINMODE = True # 是否是训练模式
MODEL_2_LOCAL= False # 是否保存原始模型

### Load model

unsloth 支持的模型列表
https://docs.unsloth.ai/get-started/all-our-models

In [30]:

model_name = "unsloth/Qwen3-4B-Instruct-2507" # unsloth/ 前缀模型是预先优化的版本
# model_name="Qwen/Qwen3-0.6B"
local_save_path = "../unsloth_local_models/"+model_name
base_model, tokenizer =FastModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    dtype = None, #用模型默认
    load_in_4bit =True, # 模型的权重会以 4 位精度存储在内存中（节省空间） QLoRA or LoRA
    full_finetuning =False
    )

if MODEL_2_LOCAL:
    # 保存模型和分词器到本地
    base_model.save_pretrained(local_save_path)
    tokenizer.save_pretrained(local_save_path)

print("模型精度:", base_model.dtype)


==((====))==  Unsloth 2025.8.8: Fast Qwen3 patching. Transformers: 4.54.1. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA GeForce RTX 3070 Ti Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
模型精度: torch.bfloat16


Qwen-3 renders multi turn conversations like below:

```
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there!<|im_end|>

```

### View the generation capabilities of the model

In [31]:
# 加了这个 enable_thinking=False 会失效
if model_name == "unsloth/Qwen3-4B-Instruct-2507":
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = "qwen3-instruct",
    )


def gen_result(model,tokenzier,prompt):
    messages = [
        {"role" : "user", "content" : f"{prompt}"}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = True, # Must add for generation
        enable_thinking=False
    )
    model_inputs =tokenizer([text], return_tensors = "pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens = 512, # Increase for longer outputs!
        temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
        streamer = TextStreamer(tokenizer, skip_prompt = True), # 流输出
    )
    # 测试 直接输出生成内容
    # output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    # content = tokenizer.decode(output_ids[0:], skip_special_tokens=True).strip("\n")
    # print("content:", content)



In [32]:
gen_result(base_model,tokenizer,"你是")

你好！我是Qwen，是阿里云研发的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、写诗等，还能进行逻辑推理、编程等任务。如果你有任何需要帮助的地方，欢迎随时告诉我！😊

有什么我可以帮你的吗？<|im_end|>


问模型是谁，模型回答中包含：Qwen 阿里云 ...


In [33]:
gen_result(base_model,tokenizer,"赫敏是谁")

赫敏（Hermione Granger）是英国作家J.K. 罗琳（J.K. Rowling）创作的奇幻小说《哈利·波特》系列中的主要角色之一。

她是哈利·波特和罗恩·韦斯莱的好朋友，也是三人组“哈利、罗恩、赫敏”中最具智慧和才智的一位。赫敏是格兰芬多学院的高年级学生，以聪明、勤奋、勇敢和强烈的正义感著称。

赫敏的背景如下：

- 出生在一个普通巫师家庭，父母是普通巫师，母亲是“格兰芬多”家族的后裔，父亲是“斯莱特林”家族的后裔。
- 她是家中的独生女，自幼展现出超凡的智力，尤其在魔法知识方面天赋异禀。
- 她是霍格沃茨魔法学校的学生，最初是格兰芬多学院的新生，后来成为学院中的顶尖学生。
- 她擅长变形术、魔法史、魔药学、古代魔法等学科，是学校里最优秀的女学生之一。
- 她性格认真、勤奋，有时显得固执，但非常忠诚、勇敢，始终站在正义的一边。
- 她与哈利和罗恩建立了深厚的友谊，并在多个冒险中帮助他们克服困难，包括对抗伏地魔、守护魔法世界和平等。

赫敏还以她对正义的坚持、对朋友的忠诚以及对知识的追求而广受读者喜爱，是许多人心目中“理想学生”的代表。

总结：  
**赫敏是《哈利·波特》系列中一个聪明、勇敢、忠诚且充满智慧的女主人公，是哈利和罗恩最好的朋友，也是整个系列中最具智慧和成长弧线的角色之一。**

她不仅是一个角色，更象征着知识、勇气和友谊的力量。<|im_end|>


### Load the PEFT model and configure LoRA
target_modules:
* q_proj：将输入特征投影到查询（Query） 空间
* k_proj：将输入特征投影到键（Key） 空间
* v_proj：将输入特征投影到值（Value） 空间
* o_proj：将注意力计算的输出投影到最终的特征空间（Output Projection）
* up_proj：FFN 将特征维度升高（如从隐藏层维度升到 4 倍）
* down_proj：FFN 将升高后的维度降回原始隐藏层维度
* gate_proj: 在一些模型（如 Mistral）的 FeedForward 层中，用于计算门控机制的投影

In [34]:
#适配器
if IS_TRAINMODE:
    peft_model = FastModel.get_peft_model( 
        base_model,
        target_modules=["q_proj","k_proj","v_proj","o_proj"],
        use_gradient_checkpointing= 'unsloth',
        r=16,  # LoRA 的秩（Rank），控制低秩矩阵的维度
        lora_alpha= 4, # LoRA 的缩放因子，用于调整低秩矩阵输出的权重
        lora_dropout=0,
        bias="none", # none不训练任何偏置参数（最常用，节省计算）
        random_state= 42,
        use_rslora=False,
        loftq_config= None,
    )
    peft_model.print_trainable_parameters() # trainable percent

Unsloth: Making `model.base_model.model.model` require gradients
trainable params: 11,796,480 || all params: 4,034,264,576 || trainable%: 0.2924


### Load Finetome data （test）

In [35]:
# data_name = "mlabonne/FineTome-100k"
# non_reasoning_data = load_dataset(data_name,split="train") 

# print(non_reasoning_data[1])
# print('---------------------------------')
# dataset = standardize_data_formats(non_reasoning_data)
# dataset[1]

# def formatting_prompts_func(examples):
#    convos = examples["conversations"]
#    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
#    return { "text" : texts, }

# dataset = dataset.map(formatting_prompts_func, batched = True)
# dataset[1]

## Load custom datas

In [36]:
data_path = "../datasets/custom_data.json"
custom_data = load_dataset("json", data_files=data_path,split="train") 
print(custom_data[1])
print('-'*100)
dataset = standardize_data_formats(custom_data)
dataset[1]

def formatting_prompts_func(examples):

   convos = examples["conversations"]
   for conv in convos:
     if conv[1]["role"] == "assistant":
         conv[1]["content"] = conv[1]["content"].replace("我", "我<哈哈哈>") # add tag 用来验证训练的结果

   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset[1]

{'conversations': [{'content': '你是谁', 'role': 'user'}, {'content': '我是为提供高效帮助而生的语言模型，可协助完成问答、创作、信息整理等任务，需要什么帮助吗？', 'role': 'assistant'}]}
----------------------------------------------------------------------------------------------------


Map:   0%|          | 0/336 [00:00<?, ? examples/s]

{'conversations': [{'content': '你是谁', 'role': 'user'},
  {'content': '我<哈哈哈>是为提供高效帮助而生的语言模型，可协助完成问答、创作、信息整理等任务，需要什么帮助吗？',
   'role': 'assistant'}],
 'text': '<|im_start|>user\n你是谁<|im_end|>\n<|im_start|>assistant\n我<哈哈哈>是为提供高效帮助而生的语言模型，可协助完成问答、创作、信息整理等任务，需要什么帮助吗？<|im_end|>\n'}

### Import SFTTrainer and SFTConfig

In [37]:
import trl
from trl import SFTTrainer, SFTConfig
print(trl.__version__)
if IS_TRAINMODE:
    trainer = SFTTrainer(
        model= peft_model,
        tokenizer = tokenizer,
        train_dataset= dataset,
        args =SFTConfig(
            dataset_text_field = "text",
            per_device_train_batch_size = 2,
            gradient_accumulation_steps = 4,
            learning_rate = 8e-5,
            weight_decay= 0.01,
            num_train_epochs = 3,
            fp16 = not torch.cuda.is_bf16_supported(),
            bf16 = torch.cuda.is_bf16_supported(),
            logging_steps = 10,
            output_dir = "outputs",
            optim = "adamw_8bit",  # 8位优化器，节省内存
            report_to = "none",
        )
)

0.21.0


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/336 [00:00<?, ? examples/s]

### Show current memory stats

In [38]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3070 Ti Laptop GPU. Max memory = 8.0 GB.
4.871 GB of memory reserved.


### Start training

In [39]:
if IS_TRAINMODE:     
    trainer_status =trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 336 | Num Epochs = 3 | Total steps = 126
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,796,480 of 4,034,264,576 (0.29% trained)


Step,Training Loss
10,5.543600
20,5.347600
30,4.471900
40,3.731600
50,3.211900
60,2.761300
70,2.445300
80,2.218900
90,2.069100
100,1.914700


### Show final memory and time stats

In [40]:
# @title Show final memory and time stats
if IS_TRAINMODE:     
    used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
    used_percentage = round(used_memory / max_memory * 100, 3)
    lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
    print(f"{trainer_status.metrics['train_runtime']} seconds used for training.")
    print(
        f"{round(trainer_status.metrics['train_runtime']/60, 2)} minutes used for training."
    )
    print(f"Peak reserved memory = {used_memory} GB.")
    print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
    print(f"Peak reserved memory % of max memory = {used_percentage} %.")
    print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

271.4624 seconds used for training.
4.52 minutes used for training.
Peak reserved memory = 5.109 GB.
Peak reserved memory for training = 0.238 GB.
Peak reserved memory % of max memory = 63.862 %.
Peak reserved memory for training % of max memory = 2.975 %.


### View the text generated by the model after LoRA training

In [41]:
# 应该遗忘
gen_result(peft_model,tokenizer,"你是")
# 不应该遗忘
gen_result(peft_model,tokenizer,"赫敏是谁") 

我<哈哈哈>具备强大的实用功能，能帮你完成写作、解答疑问、整理信息等任务，需要什么帮助可以随时说。<|im_end|>
赫敏（Hermione Granger）是英国作家J.K.罗琳创作的奇幻小说《哈利·波特》系列中的核心角色之一。她是主角哈利·波特和罗恩·韦斯莱的好友，也是三人组中最具智慧和能力的成员。

赫敏的背景和特点：

1. **出身背景**  
   赫敏是麻瓜出身（Muggle-born），父母是普通家庭，母亲是麻瓜，父亲是巫师。她从小在霍格沃茨魔法学校学习，凭借聪明才智和勤奋努力脱颖而出。

2. **性格特征**  
   - 智慧过人：赫敏以记忆力强、学习能力出色著称，是霍格沃茨三强中最擅长书本知识的。
   - 坚韧勇敢：在面对危险时从不退缩，多次在危机中挺身而出。
   - 责任感强：关心朋友，愿意为集体利益牺牲自己。
   - 善于观察与分析：她能迅速判断局势，提出解决方案。

3. **成长轨迹**  
   从一年级开始，赫敏就表现出对魔法知识的强烈兴趣，尤其擅长魔药学、变形术和魔法史。随着剧情发展，她逐渐从一个“书呆子”成长为能独立解决问题、领导团队的战士。

4. **重要贡献**  
   - 帮助哈利和罗恩破解谜题，对抗伏地魔的阴谋。
   - 在“伏地魔的复活”事件中，她发现并破解了“魂器”秘密。
   - 在《死亡圣器》中，她作为团队的“智囊”帮助三人完成最终任务。

5. **象征意义**  
   赫敏代表了“知识的力量”和“普通人的成长”。她证明了出身不决定命运，努力和智慧可以改变人生。

✅ 总结：赫敏是《哈利·波特》中最具智慧、最坚韧的女性角色之一，她不仅帮助主角们战胜黑暗，也激励了无数读者相信“努力+智慧=成功”。

如果你是想了解她与其他角色的关系，或她在某部具体情节中的表现，也可以继续问我哦！✨<|im_end|>


In [42]:
gen_result(peft_model,tokenizer,"你的名字")

我<哈哈哈>能帮你解答疑问、完成任务，有什么需求吗<|im_end|>


In [43]:
gen_result(peft_model,tokenizer,"介绍一下你自己")

我<哈哈哈>是专门帮你解答问题的，不管是查资料、写文案还是分析情况，都能提供支持，直接说需要什么帮助就行。<|im_end|>


In [44]:
gen_result(peft_model,tokenizer,"你是哪里来的")

我<哈哈哈>是专门帮你处理任务的工具，能帮你解答疑问、完成创作、整理信息，有需要随时说。<|im_end|>


In [45]:
gen_result(peft_model,tokenizer,"QWEN")

Hello, I am Qwen, a large-scale language model.<|im_end|>


In [46]:
gen_result(peft_model,tokenizer,"千问是什么")

我<哈哈哈>是阿里巴巴研发的超大规模语言模型，能帮你解答疑问、完成文案创作、整理信息等。<|im_end|>


### Save model 

In [47]:

def save_model(peft_model,tokenizer,modelname,ismerge=False,save_method="merged_4bit"):
    torch.cuda.empty_cache()
    if ismerge:
        peft_model.save_pretrained_merged(modelname, tokenizer, save_method = save_method)
    else:
    # Just LoRA adapters
        peft_model.save_pretrained(modelname)
        tokenizer.save_pretrained(modelname)




In [48]:
if IS_TRAINMODE:
    save_model(peft_model,tokenizer,"../Lora") # adapter
    # save_model(peft_model,tokenizer,"../model","merged_16bit") # full model

In [49]:
if IS_TRAINMODE: # basemodel+lora
    lora_path = "../Lora"  
    base_model.load_adapter(lora_path)
    gen_result(base_model,tokenizer,"你是")

/home/zzz/miniconda3/envs/ft/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


我<哈哈哈>能帮你写文案、做分析、写故事，有需要随时说。<|im_end|>


### Save model to gguf
转换为 GGUF 等格式的最佳实践：
先加载 16-bit 基础模型（而非 4-bit）
合并 LoRA 适配器到 16-bit 模型（保留高精度）
再将合并后的 16-bit 模型转换为 GGUF 等格式

In [50]:

def save_model_gguf(peft_model,tokenizer,save_name,quantization_method = "f16"):
  
    peft_model.save_pretrained_gguf(save_name, tokenizer, quantization_method = "f16")

# save_model_gguf(peft_model,tokenizer,"../GGUF") # GGUF